## Crop and combine images

In [3]:
import os
import numpy as np
# from util import util
import cv2
import json

%matplotlib inline

# change this path to your own path
base = '/home/cvmlserver/Seohyeon/v-coco/'

### Crop foreground image with object area


In [4]:
# path to the image folder
input_folder = os.path.join(base, 'data/person_dataset/')
mask_folder = os.path.join(base, 'data/masks/')
bg_folder = os.path.join(base, 'data/backgrounds/')
output_folder = os.path.join(base, 'data/')

female_images = os.listdir(os.path.join(input_folder, 'Female'))
male_images = os.listdir(os.path.join(input_folder, 'Male'))
bg_classes = os.listdir(bg_folder)

In [5]:
def crop_image(image, mask, image_name):
    black = np.logical_and(mask[:,:,0] != 0, np.logical_and(mask[:,:,1] != 0, mask[:,:,2] != 0))
    y, x = np.where(black)
    minx = np.min(x)
    miny = np.min(y)
    maxx = np.max(x)
    maxy = np.max(y) 
    img_trim = image[miny:maxy, minx:maxx]
    mask_trim = mask[miny:maxy, minx:maxx]
    cv2.imwrite(os.path.join(output_folder, 'cropped_images', image_name), img_trim)
    cv2.imwrite(os.path.join(output_folder, 'cropped_masks', image_name), mask_trim)

In [133]:
for i in range(len(female_images)):
    female_image = cv2.imread(os.path.join(input_folder, 'Female', female_images[i]),cv2.IMREAD_UNCHANGED)
    male_image = cv2.imread(os.path.join(input_folder, 'Male', male_images[i]),cv2.IMREAD_UNCHANGED)
    female_mask = cv2.imread(os.path.join(mask_folder, female_images[i]), cv2.COLOR_RGB2GRAY)
    male_mask = cv2.imread(os.path.join(mask_folder, male_images[i]), cv2.COLOR_RGB2GRAY)
    crop_image(female_image, female_mask, female_images[i])
    crop_image(male_image, male_mask, male_images[i])

### Combine image with area information

In [6]:
def combine_image(bg_name, image, mask, background, output_folder, image_name, loc):
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
    h, w = image.shape[:2]
    height, width = background.shape[:2]
    
    if h > w:
        new_width = w+300
        new_height = int(height * new_width / width)
        if new_height < h:
            new_height = h+100
            new_width = int(width * new_height / height)
    else:
        new_height = h+300
        new_width  = int(new_height * width / height)
        if new_width < w:
            new_width = w+100
            new_height = int(new_width * height / width)
    background = cv2.resize(background, (int(new_width), int(new_height)))
    
    mask = cv2.bitwise_not(mask)

    if loc == 'left':
        roi = background[new_height-h:, 0:w]
        masked_bg = cv2.bitwise_and(roi, mask)
        added = masked_bg + image
        background[new_height-h:, 0:w] = added
        
    elif loc == 'right':
        roi = background[new_height-h:, int(new_width-w):int(new_width)]
        masked_bg = cv2.bitwise_and(roi, mask)  
        added = masked_bg + image
        background[new_height-h:, int(new_width-w):int(new_width)] = added
    
    elif loc == 'center':
        roi = background[new_height-h:, int(new_width/2-w/2):int(new_width/2+w/2)]
        masked_bg = cv2.bitwise_and(roi, mask)
        added = masked_bg + image
        background[new_height-h:, int(new_width/2-w/2):int(new_width/2+w/2)] = added

    else:
        image = image[:int(h/3)*2, :]
        mask = mask[:int(h/3)*2, :]
        roi = background[new_height-image.shape[:2][0]:, int(new_width/2-w/2):int(new_width/2+w/2)]
        masked_bg = cv2.bitwise_and(roi, mask)
        added = masked_bg + image
        background[new_height-image.shape[:2][0]:, int(new_width/2-w/2):int(new_width/2+w/2)] = added

    if not os.path.exists(output_folder + bg_name):
        os.mkdir(os.path.join(output_folder, bg_name))

    cv2.imwrite(os.path.join(output_folder, bg_name, image_name), background)

In [7]:
input_folder = os.path.join(base, 'data/cropped_images/')
mask_folder = os.path.join(base, 'data/cropped_masks/')
bg_folder = os.path.join(base, 'data/backgrounds/')
output_folder = os.path.join(base, 'data/combined/')

with open(os.path.join(base, 'data/annotations/raw/person_image_annotation.json'), 'r') as f:
    data = json.load(f)

person_images = os.listdir(input_folder)
bg_classes = os.listdir(bg_folder)

In [9]:
for bg_class in bg_classes:
    bg_images = os.listdir(os.path.join(bg_folder, bg_class))
    for image in bg_images:
        if not image.endswith('.jpg'):
            pass
        for i in range(len(person_images)):
            person_image = cv2.imread(os.path.join(input_folder, person_images[i]))
            mask = cv2.imread(os.path.join(mask_folder, person_images[i]))
            bg = cv2.imread(os.path.join(bg_folder, bg_class, image))
            loc = data[person_images[i]][0]['cropping_info']
            combine_image(image.split('.')[0], person_image, mask, bg, output_folder, person_images[i], loc)

### Pad to masks to get the same size as the combined images

In [15]:
mask_path = os.listdir(mask_folder)
padded_folder = os.path.join(base, 'data/padded_masks/')

for bg_class in bg_classes:
    bg_images = os.listdir(os.path.join(output_folder, bg_class))
    for image in bg_images:
        combined_image = cv2.imread(os.path.join(output_folder, bg_class, image))
        
        try:
            combined_image.shape[1]
        except AttributeError:
            print("shape not found", os.path.join(output_folder, bg_class, image))
            continue

        # assert not isinstance(combined_image,type(None)), 'image not found'  
        mask_image = cv2.imread(os.path.join(mask_folder, image))
        old_image_height, old_image_width, channels = mask_image.shape
        
        new_image_width = combined_image.shape[1]
        new_image_height = combined_image.shape[0]

        color = (0,0,0)
        result = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)
        # compute center offset
        x_center = (new_image_width - old_image_width) // 2
        y_center = (new_image_height - old_image_height) // 2

        # copy img image into center of result image
        result[new_image_height-old_image_height:, 
            x_center:x_center+old_image_width] = mask_image

        # save result
        if not os.path.exists(padded_folder + bg_class):
            os.mkdir(os.path.join(padded_folder, bg_class))

        cv2.imwrite(os.path.join(padded_folder, bg_class, image), result)

shape not found /home/cvmlserver/Seohyeon/v-coco/data/combined/garage/COCO_val2014_000000008832.png


libpng error: IDAT: CRC error


AttributeError: 'NoneType' object has no attribute 'shape'